#  Analyze hotel reviews with AI in Fabric


The idea of this project is to classify the hotel reviews into a number of categories that we have predefined. We then use the review classification result to enrich the hotel filtering in the PowerBI report.

In this tutorial, you will see how to leverage **Azure AI Translator**, **Azure Text Analytics** and **Azure OpenAI** using SyanpseML in Fabric out of the box.

We will perform the task in following steps:

- Translate reviews from multiple language to English *powered by Azure AI Translator*.
- Extract key phrases from hotel reviews *powered by Azure Text Analytics*.
- Classify the reviews into four predefined categories *powered by Azure OpenAI*.
- Build PowerBI visual report via directly accessing data on LakeHouse. 


### Prerequisite

Have a lakehouse added to this notebook. You download data from a public blob, and storing that in the lakehouse.

## Load the data
First, let's load the dataset.

In [8]:
IS_CUSTOM_DATA = False  # if True, dataset has to be uploaded manually

if not IS_CUSTOM_DATA:
    # Download data files into lakehouse if it does not exist
    import os, requests

    remote_url = "https://synapseaisolutionsa.blob.core.windows.net/public/hotel_reviews"
    file_list = ["hotel_reviews_demo.csv"]
    download_path = f"/lakehouse/default/Files/hotel_reviews/raw/"

    if not os.path.exists("/lakehouse/default"):
        raise FileNotFoundError(
            "Default lakehouse not found, please add a lakehouse and restart the session."
        )
    os.makedirs(download_path, exist_ok=True)
    for fname in file_list:
        if not os.path.exists(f"{download_path}/{fname}"):
            r = requests.get(f"{remote_url}/{fname}", timeout=30)
            with open(f"{download_path}/{fname}", "wb") as f:
                f.write(r.content)
    print("Downloaded demo data files into lakehouse.")

StatementMeta(, 7241188f-d2da-49ff-b8c9-9c710a168da9, 10, Finished, Available)

Downloaded demo data files into lakehouse.


In [1]:
df = spark.read.format("csv").option("header","true").load("Files/hotel_reviews/raw/hotel_reviews_demo.csv")
# df now is a Spark DataFrame containing CSV data from "Files/hotel_reviews.csv".
display(df)

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 94aa5c73-8df5-4a8d-b05d-eff9c4554095)

## Import Packages

In [2]:
import synapse.ml.core
from synapse.ml.services import *
from pyspark.sql.functions import col, flatten, udf, lower, trim
from pyspark.sql.types import StringType

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 4, Finished, Available)

## Text Translation using Azure AI Translator 
After examining the data, it’s clear that the hotel reviews are written in various languages. To address this, I utilize Azure AI Translator to translate the reviews into English.

In [3]:
translate = (Translate()
    .setTextCol("reviews_text")
    .setToLanguage("en")
    .setOutputCol("translation")
    .setConcurrency(5))

df_en = translate.transform(df)\
        .withColumn("translation_result", flatten(col("translation.translations")))\
        .withColumn("translation", col("translation_result.text")[0])\
        .cache()

df_en = df_en.select(df_en.columns[:6]+ ["translation"])

display(df_en.tail(5))

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6bb464ff-ed74-4b9a-b3f6-222934f99b89)

## Key Phrase Extraction using Azure Text Analytics
Next, I will use Azure Text Analytics to extract key phrases from the reviews .

In [4]:
model = (AnalyzeText()
        .setTextCol("translation")
        .setKind("KeyPhraseExtraction")
        .setOutputCol("response"))

df_en_key = model.transform(df_en)\
        .withColumn("documents", col("response.documents"))\
        .withColumn("keyPhrases", col("documents.keyPhrases"))\
        .cache()

df_en_key = df_en_key.select(df_en_key.columns[:7]+ ["keyPhrases"])

display(df_en_key.tail(5))

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5cf06efd-3bf5-41bd-bd00-f6ced5c1a3e8)

## Classification using Azure OpenAI
Then, I use Azure OpenAI to classify the reviews into four predefined categories: [Service, Location, Facilities, and Sanitation]. 

In [5]:
process_column = udf(lambda x: f"Classify the following news article into 1 of the following categories: categories: [Service, Location, Facilities, Sanitation]. news article:{x},Classified categor: ", StringType())
df_en_key_prompt = df_en_key.withColumn("prompt", process_column(df_en_key["translation"])).cache()
display(df_en_key_prompt.tail(5))

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 775a253f-d319-468d-95c6-8c2c4bf74ed6)

In [6]:
deployment_name = "text-davinci-003" # deployment_name could be text-davinci-003 or code-cushman-002
completion = (
    OpenAICompletion()
    .setDeploymentName(deployment_name)
    .setMaxTokens(200)
    .setPromptCol("prompt")
    .setErrorCol("error")
    .setOutputCol("classification")
)
completed_df = completion.transform(df_en_key_prompt)\
                .withColumn("class", trim(lower(col("classification.choices.text")[0])))\
                .cache()

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 8, Finished, Available)

In [7]:
display(completed_df)

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5d55b332-7a68-4d02-b296-e1e0edd2dae5)

In [8]:
df_final = completed_df.select(completed_df.columns[:8]+["class"])
display(df_final.tail(5))

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, cbeedf5b-f2a2-4144-ad12-9b5f09301da8)

## Save data to Lakehouse and build PowerBI Report
Finally, I write the results to a Lakehouse table, and use Power BI direct lake mode to build a visual report that shows the ratings and comments of the hotels by these categories. 

Write the data into a delta table on your lakehouse:

In [9]:
delta_table_path = "Tables/hotel_review" #fill in your delta table path
df_final.write.format("delta").mode("overwrite").option('overwriteSchema', 'true').save(delta_table_path)

StatementMeta(, 0780bd80-d48a-478d-8033-85030a121d6d, 11, Submitted, Running)

## Data Cleaning
The Power BI report classification slider may display unexpected review categories, which could be caused by a phenomenon known as "hallucination" resulting from the use of a large language model.

In [10]:
df_final.select("class").distinct().show(truncate = False)

StatementMeta(, c7330c58-af84-4f83-bc2a-5aa4ea5decd5, 12, Finished, Available)

+----------------+
|class           |
+----------------+
|service.        |
|service         |
|location        |
|facilities      |
|facilities.     |
|\nlocation      |
|sanitation      |
|customer service|
+----------------+



We can add the following cell to the notebook to clean up the data:

In [14]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def translate(mapping):
    def translate_(col):
        return mapping.get(col) or col
    return udf(translate_, StringType())

mapping = {'customer service': 'service', 'service.':'service','facilities.':'facilities','\nsanitation':'sanitation','\nlocation':'location'}

#df_final.translate(mapping)("class")
df_final = df_final.withColumn("class", translate(mapping)("class"))

StatementMeta(, c7330c58-af84-4f83-bc2a-5aa4ea5decd5, 16, Finished, Available)

In [15]:
df_final.select("class").distinct().show(truncate = False)

StatementMeta(, c7330c58-af84-4f83-bc2a-5aa4ea5decd5, 17, Finished, Available)

+----------+
|class     |
+----------+
|sanitation|
|location  |
|facilities|
|service   |
+----------+



In [16]:
display(df_final)

StatementMeta(, c7330c58-af84-4f83-bc2a-5aa4ea5decd5, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7add0818-e0b0-46cf-b880-715723aa53f5)

And then rewrite it to the Lakehouse. 

In [17]:
df_final.write.format("delta").mode("overwrite").option('overwriteSchema', 'true').save(delta_table_path)

StatementMeta(, c7330c58-af84-4f83-bc2a-5aa4ea5decd5, 19, Finished, Available)